In [172]:
import pandas as pd
from glob import glob
import pickle

In [41]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
files = glob('line_items*/*')
items = []
for f in files:
    items.append(pd.read_csv(f))

In [3]:
files = glob('shipments/*')
shipments = []
for f in files:
    shipments.append(pd.read_csv(f))

In [4]:
train_df = pd.read_csv('train/train.csv')

message_df = pd.read_csv('messages/messages.csv')
actions_df = pd.read_csv('messages/actions.csv')

addresses_df = pd.read_csv('misc/addresses.csv')
user_profiles_df = pd.read_csv('misc/user_profiles.csv')
master_categories_df = pd.read_csv('misc/master_categories.csv')

sample_df  = pd.read_csv('sample_submission.csv')

merged_shipments_df = pd.concat(shipments)
merged_items_df = pd.concat(items)

In [5]:
train_df.head()

,phone_id,order_completed_at,target
0,19843,2020-01,1.0
1,19843,2020-02,0.0
2,19843,2020-03,0.0
3,19843,2020-04,0.0
4,19843,2020-05,0.0


In [6]:
train_df.describe()

,phone_id,target
count,1.159576e+06,1.159576e+06
mean,2.122805e+05,5.004027e-01
std,1.675498e+05,5.000001e-01
min,2.000000e+00,0.000000e+00
25%,7.792750e+04,0.000000e+00
50%,1.723770e+05,1.000000e+00
75%,3.100738e+05,1.000000e+00
max,7.193740e+05,1.000000e+00


In [7]:
shipments = pd.merge(merged_shipments_df, addresses_df, left_on ='ship_address_id', right_on = 'id')
shipments['order_month'] = shipments['order_completed_at'].apply(lambda x: x[:7])
train_df['order_month'] = train_df['order_completed_at']
train_with_shipments_df = pd.merge(shipments, train_df, on=['phone_id', 'order_month'])

In [8]:
# order_month
train_with_shipments_df.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at_x,shipment_starts_at,retailer,s.order_state,shipment_state,...,promo_total,total_weight,platform,os,shipped_at,id,phone_id,order_month,order_completed_at_y,target
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,...,0.0,30170,web,windows,2020-02-20 08:08:54,171906,78177,2020-02,2020-02,0.0
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,...,-150.0,11305,web,windows,2020-01-03 18:10:40,278832,96681,2020-01,2020-01,0.0
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,...,0.0,13589,app,ios,2020-02-13 12:33:53,468407,140317,2020-02,2020-02,1.0
3,905126,3404737,2471773,6278277,2020-02-12 12:39:29,2020-02-27 06:49:37,2020-02-28 05:00:00,METRO,complete,shipped,...,0.0,17580,app,ios,2020-02-28 05:52:24,3404737,140317,2020-02,2020-02,1.0
4,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,...,0.0,9726,web,mac,2020-01-25 19:55:32,61962,16,2020-01,2020-01,1.0


In [88]:
train_with_shipments_df.shape

(1552657, 25)

In [50]:
train_with_shipments_df.describe()

,user_id,ship_address_id,shipment_id,order_id,s.store_id,total_cost,rate,promo_total,total_weight,id,phone_id,target
count,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06,1.552657e+06
mean,1.243100e+06,7.093170e+06,5.796713e+06,1.004921e+07,1.306470e+02,1.281351e+02,3.084039e+00,-6.577535e+01,1.731134e+04,7.093170e+06,1.472760e+05,7.549459e-01
std,6.405497e+05,2.339686e+06,1.870865e+06,2.450648e+06,8.339042e+01,8.101631e+01,2.352854e+00,1.937663e+02,1.487902e+04,2.339686e+06,1.371394e+05,4.301194e-01
min,1.400000e+03,8.531000e+03,3.223070e+05,3.217000e+03,1.000000e+00,0.000000e+00,0.000000e+00,-2.500000e+04,0.000000e+00,8.531000e+03,2.000000e+00,0.000000e+00
25%,7.517660e+05,5.401437e+06,4.522671e+06,8.250798e+06,8.300000e+01,9.800000e+01,0.000000e+00,0.000000e+00,8.245000e+03,5.401437e+06,4.498000e+04,1.000000e+00
50%,1.299916e+06,7.400462e+06,6.073813e+06,1.027812e+07,1.230000e+02,1.580000e+02,5.000000e+00,0.000000e+00,1.362300e+04,7.400462e+06,1.056490e+05,1.000000e+00
75%,1.743510e+06,9.037081e+06,7.340688e+06,1.212049e+07,1.750000e+02,1.580000e+02,5.000000e+00,0.000000e+00,2.193000e+04,9.037081e+06,2.074500e+05,1.000000e+00
max,2.567553e+06,1.106808e+07,8.688456e+06,1.432182e+07,4.680000e+02,1.250900e+04,5.000000e+00,0.000000e+00,2.502000e+06,1.106808e+07,7.174770e+05,1.000000e+00


In [77]:
train_with_shipments_df.columns

Index(['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at_x', 'shipment_starts_at',
       'retailer', 's.order_state', 'shipment_state', 's.city_name',
       's.store_id', 'total_cost', 'rate', 'dw_kind', 'promo_total',
       'total_weight', 'platform', 'os', 'shipped_at', 'id', 'phone_id',
       'order_month', 'order_completed_at_y', 'target'],
      dtype='object')

In [109]:
grouped_single = train_with_shipments_df.groupby(['phone_id', 'order_month']).agg({'ship_address_id': ['mean', 'count'],
                                                                 'shipment_id': ['mean', 'count'], 
                                           'total_cost': ['mean', 'count']})

In [110]:
grouped_single.columns = ['ship_address_id_mean', 'ship_address_id_count', 'shipment_id_mean',
                          'shipment_id_count', 'total_cost_mean', 'total_cost_count']

In [111]:
grouped_single= grouped_single.reset_index()

In [112]:
grouped_single.head()

,phone_id,order_month,ship_address_id_mean,ship_address_id_count,shipment_id_mean,shipment_id_count,total_cost_mean,total_cost_count
0,2,2020-01,8531.0,1,2120074.0,1,98.0,1
1,2,2020-02,3127229.0,2,2414377.5,2,183.0,2
2,2,2020-03,3648206.0,1,2718666.0,1,233.0,1
3,2,2020-05,3921697.0,1,3600322.0,1,248.0,1
4,3,2020-03,4021446.0,2,2793912.0,2,0.0,2


In [113]:
grouped_single.shape

(692773, 8)

In [114]:
id_columns = ['phone_id', 'order_month']

In [115]:
grouped_single = pd.merge(grouped_single, train_df[id_columns +  ['target']], on = id_columns, how='left')

In [117]:
# grouped_single[grouped_single'target'].head()

In [118]:
grouped_single['order_month'].dtype

dtype('O')

In [119]:
grouped_single['order_month'] = pd.to_datetime(grouped_single['order_month'])

In [120]:
X_train = grouped_single[grouped_single['order_month'] < pd.to_datetime('2020-07')]
X_valid = grouped_single[grouped_single['order_month'] == pd.to_datetime('2020-07')]

In [121]:
X_train.shape, X_valid.shape, grouped_single.shape

((535492, 9), (157281, 9), (692773, 9))

In [128]:
y_train = X_train['target']
y_valid = X_valid['target']
X_train = X_train.drop(id_columns + ['target'], axis=1)
X_valid = X_valid.drop(id_columns + ['target'], axis=1)

In [122]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

In [124]:
# model = XGBClassifier(tree_method = 'gpu_hist')
# model.fit(X_train, y_train)
# preds = model.predict(X_test)
# metrics.accuracy_score(y_test, preds)

In [146]:
from sklearn.model_selection import KFold, GridSearchCV

In [166]:
cv_params = [
  {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
#     'max_depth': range(3, 10),
#     'colsample_bytree': [i/10.0 for i in range(1, 3)],
#     'gamma': [i/10.0 for i in range(3)]
  }
]    # parameters to be tries in the grid search

kfold = KFold(n_splits=10, random_state=42)
scoring = {'F1':'f1'}


fix_params = {
              'tree_method': 'gpu_hist',
              'objective':'binary:logistic'}   #other parameters, fixed for the moment 
csv = GridSearchCV(XGBClassifier(**fix_params), cv_params, scoring = 'f1', cv = kfold,  verbose=10)

In [167]:
csv.fit(X_train, y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV] learning_rate=0.01, n_estimators=50 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . learning_rate=0.01, n_estimators=50, score=0.862, total=   1.1s
[CV] learning_rate=0.01, n_estimators=50 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV] . learning_rate=0.01, n_estimators=50, score=0.831, total=   1.0s
[CV] learning_rate=0.01, n_estimators=50 .............................


...

[CV] . learning_rate=0.01, n_estimators=50, score=0.541, total=   1.0s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.868, total=   1.6s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.833, total=   1.6s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.831, total=   1.5s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.807, total=   1.2s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.810, total=   0.9s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  learning_rate=0.01, n_estimators=100, score=0.781, total=   0.9s
[CV] learning_rate=0.01, n_estimators=100 ............................
[CV]  

[CV] . learning_rate=0.1, n_estimators=150, score=0.705, total=   1.1s
[CV] learning_rate=0.1, n_estimators=150 .............................
[CV] . learning_rate=0.1, n_estimators=150, score=0.674, total=   1.9s
[CV] learning_rate=0.1, n_estimators=150 .............................
[CV] . learning_rate=0.1, n_estimators=150, score=0.541, total=   2.0s
[CV] learning_rate=0.1, n_estimators=200 .............................
[CV] . learning_rate=0.1, n_estimators=200, score=0.867, total=   2.5s
[CV] learning_rate=0.1, n_estimators=200 .............................
[CV] . learning_rate=0.1, n_estimators=200, score=0.838, total=   1.3s
[CV] learning_rate=0.1, n_estimators=200 .............................
[CV] . learning_rate=0.1, n_estimators=200, score=0.837, total=   1.2s
[CV] learning_rate=0.1, n_estimators=200 .............................
[CV] . learning_rate=0.1, n_estimators=200, score=0.816, total=   1.2s
[CV] learning_rate=0.1, n_estimators=200 .............................
[CV] .

[CV] .. learning_rate=0.3, n_estimators=50, score=0.782, total=   0.6s
[CV] learning_rate=0.3, n_estimators=50 ..............................
[CV] .. learning_rate=0.3, n_estimators=50, score=0.749, total=   0.6s
[CV] learning_rate=0.3, n_estimators=50 ..............................
[CV] .. learning_rate=0.3, n_estimators=50, score=0.705, total=   0.6s
[CV] learning_rate=0.3, n_estimators=50 ..............................
[CV] .. learning_rate=0.3, n_estimators=50, score=0.670, total=   0.6s
[CV] learning_rate=0.3, n_estimators=50 ..............................
[CV] .. learning_rate=0.3, n_estimators=50, score=0.543, total=   1.0s
[CV] learning_rate=0.3, n_estimators=100 .............................
[CV] . learning_rate=0.3, n_estimators=100, score=0.867, total=   1.5s
[CV] learning_rate=0.3, n_estimators=100 .............................
[CV] . learning_rate=0.3, n_estimators=100, score=0.838, total=   1.4s
[CV] learning_rate=0.3, n_estimators=100 .............................
[CV] .

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  3.5min finished


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=False),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method='gpu_hist',
                                     validate_paramet

In [170]:
csv.best_params_

{'learning_rate': 0.1, 'n_estimators': 50}

In [171]:
best_params = csv.best_params_
# Best XGB model that was found based on the metric score you specify
best_model = csv.best_estimator_
print(best_params)
# Save model
pickle.dump(csv.best_estimator_, open("xgb_v1.pickle", "wb"))

{'learning_rate': 0.1, 'n_estimators': 50}


NameError: name 'pickle' is not defined

In [142]:
# def f1_eval(y_pred, dtrain):
#     y_true = dtrain.get_label()
#     err = 1-f1_score(y_true, np.round(y_pred))
#     return 'f1_err', err

In [ ]:
# from sklearn import metrics
# model = XGBClassifier(tree_method = 'gpu_hist', max_depth=6, learning_rate=0.1,silent=False, objective='binary:logistic', \
#                       booster='gbtree', n_jobs=8, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, \
#                       subsample=0.8, colsample_bytree=0.8, colsample_bylevel=1, reg_alpha=0, reg_lambda=1)
# model.fit(X_train, y_train, eval_metric=f1_eval)
# preds = model.predict(X_valid)

# print(metrics.accuracy_score(y_valid, preds))
# print(metrics.f1_score(y_valid, preds))

In [173]:
from dstools.spark import init_spark2, active_namenode

clusters = ['rnd-dwh', 'rnd-pr']
namenodes = {c: active_namenode(c) for c in clusters}

session_name='hello'
arrow_enabled = True

spark = init_spark2({
    'appName': f"{session_name}",
    'spark.yarn.access.namenodes': namenodes['rnd-dwh'],
    'spark.sql.execution.arrow.enabled': arrow_enabled,
#     'spark.driver.extraClassPath': conf.get('spark.driver.extraClassPath'),
#     'spark.extraListeners': conf.get('spark.extraListeners'),
#     'spark.network.timeout': 100000
})

# sdf = spark.read.orc(f"{namenodes['rnd-dwh']}:8020/apps/hive/warehouse/dmdigpr_sb.db/ivyeliso_ngwl_train_shipments")


rnd-pr is unavailable. Please insert correct address manually.


/home/asrozhk2/.local/lib/python3.7/site-packages/dstools/spark/core.py:615: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [175]:
generate_features_df = spark.read.parquet(f"{namenodes['rnd-dwh']}:8020/apps/hive/warehouse/dmdigpr_sb.db/ivyeliso_ngwl_train_shipments").toPandas()

RuntimeError: 'MultiIndex' object has no attribute 'labels'
Note: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true. Please set it to false to disable this.

In [169]:
sample_df.head()

,Id;Predicted
0,19843;
1,471287;
2,342522;
3,457799;
4,233778;


In [ ]:
generate_features_df.head()